# IRS Project Report
## Introduction

This report will be a brief presentation on the pip-installable package of IRS, Infra-Red Simulator. The main results of the functions, along with the project challenges limitations will be demonstrated below. 

### Relevant Chemistry
Infrared (IR) spectroscopy is an analytical technique used to identify and study chemical compounds based on how they absorb infrared radiation. When molecules are exposed to IR light, specific vibrational modes of their chemical bonds are excited, depending on the frequency of the radiation.

Each functional group (like –OH, –NH₂, C=O, etc.) has characteristic absorption bands in the IR region (typically 4000–400 cm⁻¹), creating a unique molecular fingerprint. The measured wavenumbers are then plotted on an IR spectrum, with the x-axis being the frequency in wavenumbers(cm⁻¹), and the y-axis being the Intensity/Transmittance, measuring the amount of IR light absorbed on a give frequency.

Infrared spectroscopy is useful because it allows chemists to identify the presence of specific functional groups within a molecule by observing their characteristic absorption bands. It provides a quick and non-destructive method to confirm the structure of a compound. Additionally, it is commonly used to monitor chemical reactions by detecting changes in the spectrum that correspond to the formation or disappearance of bonds. Since each molecule has a unique IR absorption pattern, the technique is also valuable for distinguishing between similar compounds or verifying purity.

### Motivation
The initial project idea was to create a molecule combiner/analyzer, but was quickly dismissed due to the generality and the simplicity and the generality of the final result. Sharing interest in quantum chemistry, our team decided to create a tool combining quantum mechanical simulation and utility for chemists, which led to the idea of a computational Infra-Red Simulator. Combining vibrational spectrocopy




